In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import cactas as C
import os
import mahotas as mh
import nibabel as nib
import nrrd
import skimage.morphology
import cv2
from skimage.filters import threshold_otsu

In [3]:
import tensorflow as tf
gpu_available = tf.test.is_gpu_available()
gpu_available

2024-05-04 17:00:26.254016: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2024-05-04 17:00:28.723677: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 17:00:28.725830: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-05-04 17:00:29.020480: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-05-04 17:00:29.023030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 3

True

In [4]:
DATAPATH='/raid/mpsych/CACTAS/DATA/ESUS'
CAPATH='/raid/mpsych/CACTAS/DATA/CA24'

In [5]:
images, labels = C.Helper.load_data(DATAPATH)

In [6]:
masks = C.Helper.load_seg_data(CAPATH)

In [7]:
X_train, y_train, m_train, X_test, y_test, m_test = C.Helper.split_patients_masks(images, labels, masks)

In [8]:
X_train, y_train, m_train, X_test, y_test, m_test = C.Helper.normalization2(DATAPATH, CAPATH, X_train, y_train, m_train, X_test, y_test, m_test)

In [9]:
slice_to_patient_mapping, last_key = C.Helper.map_and_key_fulldata(y_train)

In [10]:
X_train, y_train, X_test, y_test = C.Helper.extract_slices(X_train, y_train, X_test, y_test)

(13311, 512, 512, 1) (13311, 512, 512, 1) (3455, 512, 512, 1) (3455, 512, 512, 1)


In [11]:
m_train, m_test = C.Helper.extract_masks_slices(m_train, m_test)

(13311, 512, 512, 1) (3455, 512, 512, 1)


In [12]:
train_masks=[]
for i in range(len(m_train)):
    binary = (m_train[i] > 0).astype(np.uint8)
    train_masks.append(binary)

In [13]:
train_images=[]
for i in range(len(X_train)):
    train_image = X_train[i] * train_masks[i]
    train_images.append(train_image)
train_images_array = np.array(train_images)

In [14]:
#train_images_array = train_images_array.astype(np.float32)

In [15]:
train_images_array = train_images_array.reshape(train_images_array.shape[0], train_images_array.shape[1],train_images_array.shape[2], 1)

In [16]:
# for i in range(100, 200):
#     plt.imshow(train_images[i])
#     plt.show()

In [17]:
# for i in range(100, 200):
#     plt.imshow(train_images[i], vmin=np.min(train_images[i][train_images[i] > 0]), vmax=np.max(train_images[i]))
#     plt.imshow(y_train[i], cmap='gray', alpha=0.5)
#     plt.show()

In [18]:
test_masks=[]
for i in range(len(m_test)):
    binary = (m_test[i] > 0).astype(np.uint8)
    test_masks.append(binary)

In [19]:
test_images=[]
for i in range(len(X_test)):
    test_image = X_test[i] * test_masks[i]
    test_images.append(test_image)
test_images_array = np.array(test_images)

In [20]:
#test_images_array = test_images_array.astype(np.float32)

In [21]:
test_images_array = test_images_array.reshape(test_images_array.shape[0], test_images_array.shape[1],test_images_array.shape[2], 1)

In [22]:
X_train_set = train_images_array[:last_key]
y_train_set = y_train[:last_key]
X_val = train_images_array[last_key:]
y_val = y_train[last_key:]

In [ ]:
y_train_set = y_train_set.astype(np.float64)

In [ ]:
y_val = y_val.astype(np.float64)

In [ ]:
X_train_set.dtype, y_train_set.dtype

In [ ]:
X_val.dtype, y_val.dtype

In [ ]:
X_train[0].dtype, y_train[0].dtype

In [ ]:
X_train_set.shape, y_train_set.shape, X_val.shape, y_val.shape

In [ ]:
X_train_set[0].shape

In [ ]:
num_samples = len(X_train_set)
num_channels = 1  # Adjust based on your combined image channels
height, width = 512, 512
dtype_size = np.float64(0).itemsize  # Change if using a different data type

memory_per_image = height * width * num_channels * dtype_size
total_memory = num_samples * memory_per_image
print(f"Estimated memory per image: {memory_per_image / 1e6} MB")
print(f"Estimated total memory for training set: {total_memory / 1e9} GB")

In [ ]:
#64
#
#

In [ ]:
#32 
#Estimated memory per image: 1.048576 MB
#Estimated total memory for training set: 11.423186944 GB

In [ ]:
#16
#Estimated memory per image: 0.524288 MB
#Estimated total memory for training set: 5.618270208 GB

In [33]:
model = C.Helper.create_unet(X_train_set[0].shape)

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


2024-05-04 16:59:03.458163: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-05-04 16:59:03.460903: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-05-04 16:59:03.469140: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0, 1
2024-05-04 16:59:03.473082: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2024-05-04 16:59:03.476256: I tensorflow/core

In [ ]:
model, history = C.Helper.train_unet(X_train_set, y_train_set, X_val, y_val, model, epochs=200)

In [ ]:
C.Helper.visualize_graph(history)

In [ ]:
y_pred = C.Helper.prediction(test_images_array, model)

In [ ]:
C.Helper.visualize_result(test_images_array, y_test, y_pred)